In [74]:
from z3 import *
import math
import time

In [75]:
def encode(U, V, k):
    
    n = len(U)
    
    m = max([len(u) for u in U])
    m = max(m, max([len(v) for v in V]))
    
    l = []
    for string in U:
        for i in range(len(string)):
            l += [int(string[i])]
    for string in V:
        for i in range(len(string)):
            l += [int(string[i])]
    s = max(l)
    
    # u[i] denotes the binary encoding of the i-th character of the word u
    u = [[Bool(f'u_{i}_{p}') for p in range(1, 1 + math.floor(math.log2(s)) + 1)] for i in range(1, k*m + 1)]
    v = [[Bool(f'v_{i}_{p}') for p in range(1, 1 + math.floor(math.log2(s)) + 1)] for i in range(1, k*m + 1)]
    
    # pu[i] denotes the start position in the word u of the i-th string
    #Note: we use max(2, ...) for convenience, because otherwise for j=1 pu[j-1] would be empty
    pu = [[Bool(f'pu_{j}_{p}') for p in range(1, max(2, math.floor(math.log2((j-1)*m + 1)) + 2))] for j in range(1, k+2)]
    pv = [[Bool(f'pv_{j}_{p}') for p in range(1, max(2, math.floor(math.log2((j-1)*m + 1)) + 2))] for j in range(1, k+2)]
    
    # ind[i] denotes the index of the i-th tile in the solution
    ind = [[Bool(f'ind_{i}_{p}') for p in range(1, math.floor(math.log2(n)) + 2)] for i in range(1, k+1)]
    
    return u, v, pu, pv, ind, m, s
    

# function that receives as input a list of propositional variables and an integer
# and outputs a boolean formula encoding that integer in binary:
def equals(l, i):
    
    b = bin(i)[2:]
    
    while len(b) < len(l):
        b = '0' + b
    
    res = True
    
    for j in range(len(l)):
        if b[j] == '0':
            res = And(res, Not(l[j]))
        elif b[j] == '1':
            res = And(res, l[j])
            
    return simplify(res)


#returns the integer corresponding to the x-th symbol of the string a_i
def at(a_i, x):
    
    return int(a_i[x-1])
        

#return a formula stating the equivalence of u_i and v_i
def equiv(u_i, v_i):
    
    res = True
    for j in range(len(u_i)):
        res = And(res, u_i[j]==v_i[j])
    return simplify(res)

In [101]:
def bpcp(U, V, k):
    
    #start time:
    st = time.time()
    
    S = Solver()
    
    u, v, pu, pv, ind, m, s = encode(U, V, k)
    
    
    def word(j, A, uu, pa, ind):
    
        res = False
        for i in range(1, len(A) + 1):

            aux = False
            a_i = A[i-1]

            for p in range(j, (j-1)*m + 2):

                a = equals(pa[j-1], p)
                b = equals(pa[j], p + len(a_i))
                c = True

                for x in range(1, len(a_i) + 1):
                    c = And(c, equals(uu[p+x-2], at(a_i, x)))

                aux = Or(aux, And(a, b, c))

            res = Or(res, And(equals(ind[j-1], i), aux))
        
        return simplify(res)
    
    
    def vectorsEq(l, k):
    
        res = True
        for i in range(1, l + 1):
            res = And(res, equiv(u[i-1], v[i-1]))
            
        return simplify( And(res, equals(pu[0], 1), equals(pv[0], 1), equals(pu[k], l + 1), equals(pv[k], l + 1)) )
    
    
    formula = False
    
    for l in range(k, k*m + 1):
        
        aux = True
        for j in range(1, k + 1):
            aux = And(aux, And(word(j, U, u, pu, ind), word(j, V, v, pv, ind)))
        
        formula = Or( formula, And(vectorsEq(l, k), aux) )
        
    S.add(simplify(formula))
    
    EncodingTime = (time.time()-st)
        
    if S.check()==sat:
        
        M = S.model()
        
        #aux is a list of pairs [i,j] such that ind_i_j is satisfied by M
        aux = [ [int(a) for a in (p.name()).split('_')[1:]]  for p in M.decls() if M[p] and p.name()[0:3]=='ind']
        
        #sol will be the solution: list of indexes of tiles
        sol = [0 for i in range(1, k+1)]
        
        b = len(ind[0]) #number of binary digits
        
        for a in aux:
            sol[a[0]-1] += 2**(b - a[1])
        
        TS = [U[j-1] for j in sol]
        BS = [V[j-1] for j in sol]
        print('Top: ' + str(TS))
        print('Bottom: ' + str(BS) + '\n')
        
        
        print('Encoding time = ' + str(EncodingTime))
        print('Full execution time = ' + str(time.time()-st))
        return sol
        
    else:
        print('Encoding time = ' + str(EncodingTime))
        print('Full execution time = ' + str(time.time()-st))
        return unsat